In [1]:
class UserFeats(object):

    def __init__(
        self
    ):
        self._ans_cnt, self._ans_corr_cnt = 0, 0

    def get_ans_cnt(self):
        return self._ans_cnt

    def get_ans_corr_cnt(self):
        return self._ans_corr_cnt

    def incr_ans_cnt(self, val):
        self._ans_cnt += val

    def incr_ans_corr_cnt(self, val):
        self._ans_corr_cnt += val

In [27]:
class QuesFeats(object):
    def __init__(self):
        self.acc = 1
        self.cnt = 2
        self.corr_cnt =1
#         self.acc = feats_tuple[self.FEAT_KEY_TO_INDEX[self.ACC]]
#         self.cnt = feats_tuple[self.FEAT_KEY_TO_INDEX[self.CNT]]
#         self.corr_cnt = feats_tuple[self.FEAT_KEY_TO_INDEX[self.CORR_CNT]]

In [4]:
user_dic = {}
user = UserFeats()

In [13]:
user.incr_ans_cnt(1)

In [22]:
user.incr_ans_corr_cnt(1)

In [7]:
user_dic[123] = user

In [14]:
user_dic[123].get_ans_cnt()

2

In [29]:
qs = QuesFeats()

In [30]:
import pandas as pd
train = pd.read_feather('../data/input/train.feather')

In [71]:
name_to_extract_fn = {
    "user_ans_cnt": lambda u_feats, q_feats, row, cache: u_feats.get_ans_cnt(),
    "user_ans_corr_cnt": lambda u_feats, q_feats, row, cache: u_feats.get_ans_corr_cnt(),
    "ques_acc": lambda u_feats, q_feats, row, cache: q_feats.acc,
    "ques_cnt": lambda u_feats, q_feats, row, cache: q_feats.cnt,
    "ques_corr_cnt": lambda u_feats, q_feats, row, cache: q_feats.corr_cnt,
}

In [73]:
feat_names = ['user_ans_cnt','user_ans_corr_cnt','ques_acc','ques_cnt','ques_corr_cnt']

feat_extract_fns = [name_to_extract_fn[name] for name in feat_names]

In [68]:
["ques_acc": lambda u_feats, q_feats, row, cache: q_feats.acc]

SyntaxError: invalid syntax (<ipython-input-68-8d034ef27cdd>, line 1)

In [53]:
for i,row in train[:90].iterrows():
    row

In [54]:
row

row_id                                89
timestamp                         653762
user_id                             2746
content_id                          6808
content_type_id                        1
task_container_id                     14
user_answer                           -1
answered_correctly                    -1
prior_question_elapsed_time          NaN
prior_question_had_explanation     False
Name: 89, dtype: object

In [70]:
# feat_names = ['user_ans_cnt','user_ans_corr_cnt','ques_acc','ques_cnt','ques_corr_cnt']
vv = []

vv.append([fn(user,qs,row,qs) for fn in feat_extract_fns])

NameError: name 'is_valid' is not defined

In [64]:
vv

[[2, 1, 1, 2, 1]]

In [24]:
len(feat_extract_fns)

5

In [19]:
[fn() for fn in name_to_extract_fn]

TypeError: 'str' object is not callable

In [ ]:
class FeatureEngineer(object):
    def __init__(self, user_feats, ques_feats, elapsed_time_mean, feat_names):
        self._user_feats = user_feats
        self._ques_feats = ques_feats
        self._elapsed_time_mean = elapsed_time_mean
        self._init_feat_extact_fns(feat_names)

    def _init_feat_extact_fns(self):
        name_to_extract_fn = {
            "prior_question_elapsed_time": lambda u_feats, q_feats, row, cache: row[
                "prior_question_elapsed_time"
            ]
            if is_valid(row["prior_question_elapsed_time"])
            else self._elapsed_time_mean,
            "prior_question_had_explanation": lambda u_feats, q_feats, row, cache: 1
            if (row["prior_question_had_explanation"] is True)
            or (row["prior_question_had_explanation"] == 1)
            else 0,
            "user_ans_cnt": lambda u_feats, q_feats, row, cache: u_feats.get_ans_cnt(),
            "user_ans_corr_cnt": lambda u_feats, q_feats, row, cache: u_feats.get_ans_corr_cnt(),
            "ques_acc": lambda u_feats, q_feats, row, cache: q_feats.acc,
            "ques_cnt": lambda u_feats, q_feats, row, cache: q_feats.cnt,
            "ques_corr_cnt": lambda u_feats, q_feats, row, cache: q_feats.corr_cnt,
        }
        self._feat_extract_fns = [name_to_extract_fn[name] for name in feat_names]


    def get_feat_values_and_pre_labels(
        self, df: DataFrame, update_per_row: bool = False
    ) -> Tuple[np.ndarray, List[int]]:
        """
        Get feature values and previous labels based on test_df
        """
        pre_labels = ... # extract previous labels
        # update if necessary
        fvs = []
        for i, row in df.iterrows():
            # skip lecture
            u_feats = self._user_feats[row["user_id"]]
            q_feats = self._ques_feats[row["content_id"]]
            fvs.append(self._get_row_fvs(row, u_feats, q_feats))


        return np.array(fvs), pre_labels


    def _get_row_fvs(self, row: Series, u_feats: UserFeats, q_feats: QuesFeats) -> List[float]:
         cache = ... # extract stats that is used by multiple features
         return [
            fn(u_feats, q_feats, row, cache) for fn in self._feat_extract_fns
        ]